In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from torch_geometric.data import Data, DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [6]:
!rm QG_jets.npz
!wget "https://zenodo.org/records/3164691/files/QG_jets.npz?download=1" -O QG_jets_new.npz


--2025-04-02 05:15:31--  https://zenodo.org/records/3164691/files/QG_jets.npz?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106689379 (102M) [application/octet-stream]
Saving to: ‘QG_jets_new.npz’

QG_jets_new.npz     100%[===================>] 101.75M  12.1MB/s    in 9.7s    

2025-04-02 05:15:42 (10.5 MB/s) - ‘QG_jets_new.npz’ saved [106689379/106689379]



In [15]:
data_path = '/content/QG_jets_new.npz'
data = np.load(data_path, allow_pickle=True)

X = data['X']
y = data['y']

# Define a function to construct edge indices based on proximity
def construct_edge_index(particle_features, k=3):
    """
    Constructs edge indices for a k-nearest neighbors graph.
    particle_features: np.array of shape (num_particles, num_features)
    k: number of nearest neighbors
    """
    from sklearn.neighbors import NearestNeighbors
    num_particles = particle_features.shape[0]
    if num_particles < k + 1:
        k = num_particles - 1
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(particle_features)
    distances, indices = nbrs.kneighbors(particle_features)
    edge_index = []
    for i in range(num_particles):
        for j in range(1, k+1):
            edge_index.append((i, indices[i, j]))
    edge_index = np.array(edge_index).T
    return torch.tensor(edge_index, dtype=torch.long)

class ParticleJetDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float)
        edge_index = construct_edge_index(x[:, :3])
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = ParticleJetDataset(X_train, y_train)
test_dataset = ParticleJetDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [16]:
from torch_geometric.utils import dense_to_sparse
from torch.utils.data import random_split

class ParticleJetDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = torch.tensor(y, dtype=torch.long)
        self.num_graphs = X.shape[0]

    def __len__(self):
        return self.num_graphs

    def __getitem__(self, idx):
        node_features = torch.tensor(self.X[idx], dtype=torch.float)
        label = self.y[idx]

        edge_index = torch.combinations(torch.arange(node_features.shape[0]), r=2).T
        edge_index = edge_index.to(torch.long)

        return Data(x=node_features, edge_index=edge_index, y=label, num_nodes=node_features.shape[0])

dataset = ParticleJetDataset(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training Samples: {len(train_dataset)}, Testing Samples: {len(test_dataset)}")


Training Samples: 80000, Testing Samples: 20000


In [17]:
from torch_geometric.nn import global_mean_pool

# class GCN(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv2(x, edge_index)
#         return F.log_softmax(x, dim=1)
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=4, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * 4, hidden_channels, heads=1, dropout=0.6)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [24]:
# def train_model(model, train_loader, optimizer):
#     model.train()
#     total_loss = 0
#     for data in train_loader:
#         optimizer.zero_grad()
#         output = model(data.x, data.edge_index)
#         loss = F.nll_loss(output[data.batch], data.y)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     return total_loss / len(train_loader)

# def evaluate_model(model, test_loader):
#     model.eval()
#     correct = 0
#     for data in test_loader:
#         output = model(data.x, data.edge_index)
#         pred = output.argmax(dim=1)
#         correct += pred.eq(data.y).sum().item()
#     return correct / len(test_loader.dataset)
def train_model(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)

        output = model(data.x, data.edge_index, data.batch)
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)
    return correct / total


# # Choose model: 'GCN' or 'GAT'
# model_choice = 'GCN'

# if model_choice == 'GCN':
#     model = GCN(in_channels=4, hidden_channels=32, out_channels=2)
# else:
#     model = GAT(in_channels=4, hidden_channels=32, out_channels=2)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Train and Evaluate
# num_epochs = 10
# for epoch in range(num_epochs):
#     loss = train_model(model, train_loader, optimizer)
#     accuracy = evaluate_model(model, test_loader)
#     print(f"Epoch {epoch + 1}: Loss = {loss:.4f}, Accuracy = {accuracy * 100:.2f}%")

# print("Training complete!")


In [ ]:
num_features = X.shape[-1]
models = {'GCN': GCN(num_features, 32, 2), 'GAT': GAT(num_features, 32, 2)}

results = {}
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for name, model in models.items():
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        loss = train_model(model, train_loader, optimizer)
        acc = evaluate_model(model, test_loader)
        print(f"{name} - Epoch {epoch + 1}: Loss = {loss:.4f}, Accuracy = {acc * 100:.2f}%")

    final_acc = evaluate_model(model, test_loader)
    results[name] = final_acc * 100

print("\n=== Final Model Performance ===")
for name, acc in results.items():
    print(f"{name}: {acc:.2f}% Accuracy")


GCN - Epoch 1: Loss = 0.6784, Accuracy = 71.60%
GCN - Epoch 2: Loss = 0.5876, Accuracy = 70.06%
GCN - Epoch 3: Loss = 0.5712, Accuracy = 73.75%
GCN - Epoch 4: Loss = 0.5594, Accuracy = 73.64%
GCN - Epoch 5: Loss = 0.5435, Accuracy = 73.98%
GCN - Epoch 6: Loss = 0.5361, Accuracy = 74.63%
GCN - Epoch 7: Loss = 0.5298, Accuracy = 74.05%
GCN - Epoch 8: Loss = 0.5258, Accuracy = 74.61%
GCN - Epoch 9: Loss = 0.5229, Accuracy = 75.16%
GCN - Epoch 10: Loss = 0.5227, Accuracy = 75.67%
GCN - Epoch 11: Loss = 0.5208, Accuracy = 75.21%
GCN - Epoch 12: Loss = 0.5188, Accuracy = 74.78%
GCN - Epoch 13: Loss = 0.5171, Accuracy = 75.14%
GCN - Epoch 14: Loss = 0.5154, Accuracy = 75.12%
GCN - Epoch 15: Loss = 0.5135, Accuracy = 74.59%
GCN - Epoch 16: Loss = 0.5138, Accuracy = 75.14%
GCN - Epoch 17: Loss = 0.5132, Accuracy = 75.79%
GCN - Epoch 18: Loss = 0.5129, Accuracy = 75.47%
GCN - Epoch 19: Loss = 0.5125, Accuracy = 75.62%
GCN - Epoch 20: Loss = 0.5105, Accuracy = 75.73%
GCN - Epoch 21: Loss = 0.5114